<a href="https://colab.research.google.com/github/cerr/pycerr-notebooks/blob/main/autosegment_MR_Prostate_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the CTV and OARs on T2w-MRI prostate scans.

## Requirements
* Python>=3.8
* Applying this model requires access to a GPU.  
  *On Colab* :  `Runtime > Change runtime type > Select GPU `

## AI model
* The segmentation model used here was trained and validated on T2w-MR scans used for RT planning. Its performance on diagnostic scans is expected to be sub-optimal.
* The trained model is packaged as a Conda environment archive containing  python libraries and other dependencies.

## I/O
* **Input**: DICOM-format T2w-MR scan(s) of the prostate.  
  
Input data should be organized as: one directory of DICOM images per patient.    
  
    
    Input dir
            |------Pat1  
                      |------img1.dcm  
                             img2.dcm  
                             ....  
                             ....  
            |-----Pat2  
                     |------img1.dcm  
                            img2.dcm  
                            ....  
                            ....  


* **Output**: DICOM RTStruct-format segmentations.


## Running the model

* The Conda archive is downloaded to a configurable `condaEnvPath`. By default `condaEnvPath = '/content/pretrainedModel/'`
* The inference script is located at   
`wrapperPath = os.path.join(condaEnvPath,'MR_Prostate_Deeplab/model_wrapper/run_inference_nii.py')`

* Command to execute the model
```python
!python {wrapperPath} {input_nii_directory} {output_nii_directory}
```
* [***pyCERR***](https://github.com/cerr/pyCERR) is used for data pre- and post-processing, including converting DICOM to NIfTI format, required to run the model.

## License

By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

**`THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.`**

`This software is for research purposes only and has not been approved for clinical use.`

`Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.`
  
`YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.`

`All Technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain
such licenses to export, re-export, or import as may be required after delivery to you.`

**`You may publish papers and books using results produced using software provided that you reference the appropriate citations`**:
  
  * ***AI model***: https://doi.org/10.1016/j.phro.2019.11.006
  * ***CERR model library***: https://doi.org/10.1016/j.ejmp.2020.04.011



# Downloads


## Download MR images (DICOM) from ***dataUrl*** to ***dataDownloadDir***


In [ ]:
import os
workDir = r'/home/jupyter' # r'/content' for Colab
dataUrl = 'http://path.to/data'
dataDownloadDir = os.path.join(workDir,'sampleData','testMRdicom')
os.makedirs(dataDownloadDir,exist_ok=True)

In [ ]:
! wget -O sampleData.gz -L {dataUrl}
! tar xf sampleData.gz -C {dataDownloadDir}
! rm sampleData.gz

Define paths to input DICOM directory, desired output directory, and a session directory to store temporary files during model execution

In [ ]:
#Paths to input data and conda env with pre-trained models
inputDicomPath = os.path.join(dataDownloadDir,'your_dir_name_here')  # Replace with apropriate path to your dataset
outputDicomPath = os.path.join(workDir, 'AIoutput')
sessionPath = os.path.join(workDir, 'temp')

if not os.path.exists(outputDicomPath):
  os.mkdir(outputDicomPath)

if not os.path.exists(sessionPath):
  os.mkdir(sessionPath)

## Download pre-packaged Conda archive to ***condaArchiveDir***



In [ ]:
%%capture
# Location of LungOAR archive
condaArchiveDir = os.path.join(workDir, 'pretrainedMRProstateModel')

# Download packaged environment for the AI model
boxHash = r'H4sIAPu6TGYAAwXBSQ6AIAwAwLuPofEgMf6mrSxGEaRVkdc7E1WLLABJdmZDuRnOCSRidSuIom4MbG+H5dA40smPTV+gNtWL3OxfXwhN6MMPPP87xkgAAAA='
saveFileName = 'mr_prostate_seg.tar.gz'

!mkdir -p {condaArchiveDir}
!wget `base64 -d <<<{boxHash} | gunzip` -O {saveFileName}
!tar xf {saveFileName} -C {condaArchiveDir}
!rm {saveFileName}

In [ ]:
%%capture
# Path to activation script for Conda environment
activateScript = condaArchiveDir +'/bin/activate'

# Path to AI model inference script
wrapperPath = os.path.join(condaArchiveDir,'MR_Prostate_Deeplab', \
                          'model_wrapper','run_inference_nii.py')

## Install ***pyCERR***

pyCERR is used for data import/export and pre- and post-processing transformations needed for this model.

In [ ]:
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR"

# Data processing

## ***Pre-processing***: Identify the input scan

In [ ]:
import cerr
from cerr.dataclasses import structure
from cerr.contour import rasterseg as rs
from cerr.utils import identifyScan, imageProc

def identifyInputData(planC):

  # Identify scan index in  planC
  scanIdS = {"imageType": "MR SCAN"}
  matchScanIdx = identifyScan.getScanNumForIdentifier(scanIdS, planC,
                                                    False)

  return matchScanIdx


## ***Post-processing***: Import label maps to planC

In [ ]:
# Map output labels to structure names
strToLabelMap = {1:"Bladder_O_DLV3", 2:"CTV_PROST_DLV3", 3:"Penile_Bulb_DLV3",
                 4:"Rectum_O_DLV3", 5:"Urethra_Foley_DLV3",\
                 6:"Rectal_Spacer_DLV3", 7: "Bowel_Lg_DLV3"}
numLabel = len(strToLabelMap)

In [ ]:
#Import label map to CERR
import glob

def postProcAndImportSeg(outputDir,scanNum,planC):
  niiGlob = glob.glob(os.path.join(outputDir,'*.nii.gz'))

  print('Importing ' + niiGlob[0]+'...')
  numStrOrig = len(planC.structure)
  planC = pc.load_nii_structure(niiGlob[0], scanNum, planC, \
                              labels_dict = strToLabelMap)
  return planC

# Segment OARs

## Apply the model to all CT scans in ***inputDicomPath*** and store auto-segmentation results to ***outputDicomPath***

In [ ]:
%%capture
import subprocess
import numpy as np
import cerr
from cerr import plan_container as pc
from cerr.dataclasses import scan as cerrScn
from cerr.utils.aiPipeline import createSessionDir as cdir
from cerr.dcm_export import rtstruct_iod

# Loop over pyCERR files
fileList = os.listdir(inputDicomPath)
numFiles = len(fileList)
modality = 'MR'

for iFile in range(numFiles):

    dcmDir = os.path.join(inputDicomPath,fileList[iFile])

    # Create session dir to store temporary data
    modInputPath, modOutputPath = cdir.createSessionDir(sessionPath,
                                                        inputDicomPath)

    # Import DICOM scan to planC
    planC = pc.load_dcm_dir(dcmDir)
    scanNum = identifyInputData(planC)
    numExistingStructs = len(planC.structure)

    # Export inputs to NIfTI
    scanFilename = os.path.join(modInputPath,
                                f"{fileList[iFile]}_scan_3D.nii.gz")
    planC.scan[scanNum].save_nii(scanFilename)

    # Apply model
    subprocess.run(f"source {activateScript} && python {wrapperPath} \
                  {modInputPath} {modOutputPath}", \
                  capture_output=False,shell=True,executable="/bin/bash")

    # Import results to planC
    planC = postProcAndImportSeg(modOutputPath,scanNum,planC)
    newNumStructs = len(planC.structure)

    # Export segmentations to DICOM
    structFileName = fileList[iFile]+'_AI_seg.dcm'
    structFilePath = os.path.join(outputDicomPath,structFileName)
    structNumV = np.arange(numExistingStructs,newNumStructs)
    indOrigV = np.array([cerrScn.getScanNumFromUID(planC.structure[structNum].assocScanUID,\
                        planC) for structNum in structNumV], dtype=int)
    structsToExportV = structNumV[indOrigV == scanNum]
    seriesDescription = "AI Generated"
    exportOpts = {'seriesDescription': seriesDescription}
    rtstruct_iod.create(structsToExportV,structFilePath,planC,exportOpts)

## **Optional**: Uncomment the following to download the output segmentations to your workspace bucket.

In [ ]:
# workspaceBucket = os.environ['WORKSPACE_BUCKET']
# !gcloud storage cp -r {outputDicomPath} {workspaceBucket}

# Display results

## Overlay AI segmentations on scan for visualization using ***Matplotlib***

Note: This example displays the last segmented dataset by default.  
Load the appropriate pyCERR archive to `planC` to view results for desired dataset.

In [ ]:
from cerr.viewer import showMplNb

showMplNb(scanNum, structsToExportV, planC,\
          windowCenter=1100, windowWidth=2250)